## Основы скрапинга и парсинга
*[По материалам](https://proglib.io/p/samouchitel-po-python-dlya-nachinayushchih-chast-17-osnovy-skrapinga-i-parsinga-2023-03-13)*
### Скрапинг содержимого страницы
Воспользуемся модулем urllib.request стандартной библиотеки urllib для получения исходного кода ОДНОСТРАНИЧНОГО САЙТА example.com

In [ ]:
from urllib.request import urlopen
url = 'http://example.com'
page = urlopen(url)
print(page.read().decode('utf-8'))

Точно такой же результат можно получить с помощью requests:

In [8]:
from bs4 import BeautifulSoup
import requests
url = 'http://example.com'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')
print(soup)

<!DOCTYPE html>

<html>
<head>
<title>Example Domain</title>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    @media (max-width: 700px) {
        div {
            margin: 0 auto;
            width: auto;
        }
    }
    </style>
</head>
<body>
<div>
<h1>Example Domain</h1>
<p>This domain is for use in illustrative example

### Парсинг полученных данных
### Способы извлечения данных:
- Метод строк
- Регулярне выражения
- Запрос XPath
- Обработка BeautifulSoup

ИЗВЛЕЧЕМ АДРЕС ССЫЛКИ
### Метод строк (самый трудоемкий)

In [10]:
from urllib.request import urlopen
url = 'http://example.com/'
page = urlopen(url)
html_code = page.read().decode('utf-8')
start = html_code.find('href="') + 6
end = html_code.find('">More')
link = html_code[start:end]
print(link)

https://www.iana.org/domains/example


### Применение регулярных выражений

In [9]:
from urllib.request import urlopen
import re
url = 'http://example.com/'
page = urlopen(url)
html_code = page.read().decode('utf-8')
link = r'(https?://\S+)(?=")'
print(re.findall(link, html_code))

['https://www.iana.org/domains/example']


### Запрос XPath
Язык запросов XPath (XML Path Language) позволяет извлекать данные из определенных узлов XML-документа. Для работы с HTML кодом в Python используют модуль etree:

In [11]:
from urllib.request import urlopen
from lxml import etree
url = 'http://example.com/'
page = urlopen(url)
html_code = page.read().decode('utf-8')
tree = etree.HTML(html_code)
print(tree.xpath("/html/body/div/p[2]/a/@href")[0])

https://www.iana.org/domains/example


Чтобы узнать путь к нужному элементу страницы, в браузерах Chrome и FireFox надо кликнуть правой кнопкой по элементу и выбрать «Просмотреть код», после чего откроется консоль. В консоли по интересующему элементу нужно еще раз кликнуть правой кнопкой, выбрать «Копировать», а затем – копировать путь XPath:
![](https://media.proglib.io/posts/2023/03/08/1c7d748f356f2bc01980724eb8a754c1.png)

В приведенном выше примере для извлечения ссылки к пути `/html/body/div/p[2]/a/` мы добавили указание для получения значения ссылки `@href`, и индекс `[0]`, поскольку результат возвращается в виде списка. Если `@href` заменить на `text()`, программа вернет текст ссылки, а не сам URL:
```python
print(tree.xpath("/html/body/div/p[2]/a/text()")[0])
```

### Парсинг с применением BeautiulSoup
Библиоеку необходимо установить
```
pip install beautifulsoup4
```
В приведенном ниже примере мы будем извлекать из исходного кода страницы уникальные ссылки, за исключением внутренних:

In [12]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
url = 'https://webscraper.io/test-sites/e-commerce/allinone/phones'
page = urlopen(url)
html = page.read().decode('utf-8')
soup = BeautifulSoup(html, 'html.parser')
links = set()
for link in soup.find_all('a'):
    l = link.get('href')
    if l != None and l.startswith('https'):
        links.add(l)
for link in links:
    print(link)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)>

При использовании XPath точно такой же результат даст следующий скрипт:

In [7]:
from urllib.request import urlopen
from lxml import etree
url = 'https://webscraper.io/test-sites/e-commerce/allinone/phones'
page = urlopen(url)
html_code = page.read().decode('utf-8')
tree = etree.HTML(html_code)
sp = tree.xpath("//li/a/@href")
links = set()
for link in sp:
    if link.startswith('http'):
        links.add(link)
for link in links:
    print(link)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)>

### Имитация действий пользователя в браузере
При скрапинге сайтов очень часто требуется авторизация, нажатие кнопок «Читать дальше», переход по ссылкам, отправка форм, прокручивание ленты и так далее. Отсюда возникает необходимость имитации действий пользователя. Как правило, для этих целей используют `Selenium`, однако есть и более легкое решение – библиотека `MechanicalSoup`:
```
pip install MechanicalSoup
```
По сути, MechanicalSoup исполняет роль браузера без графического интерфейса. Помимо имитации нужного взаимодействия с элементами страниц, MechanicalSoup также парсит HTML-код, используя для этого все функции BeautifulSoup.

Воспользуемся тестовым сайтом http://httpbin.org/, на котором есть возможность отправки формы заказа пиццы:

In [ ]:
import mechanicalsoup
browser = mechanicalsoup.StatefulBrowser()
browser.open("http://httpbin.org/")
browser.follow_link("forms")
browser.select_form('form[action="/post"]')
print(browser.form.print_summary())

В приведенном выше примере браузер MechanicalSoup перешел по внутренней ссылке http://httpbin.org/forms/post и вернул описание полей ввода.
Перейдем к имитации заполнения формы:

In [ ]:
browser["custname"] = "Best Customer"
browser["custtel"] = "+7 916 123 45 67"
browser["custemail"] = "trex@example.com"
browser["size"] = "large"
browser["topping"] = ("cheese", "mushroom")
browser["comments"] = "Add more cheese, plz. More than the last time!"

Теперь форму можно отправить:

In [ ]:
response = browser.submit_selected()

In [ ]:
# Результат можно вывести с помощью 
print(response.text)

### Скрапинг и парсинг динамического контента

Все примеры, которые мы рассмотрели выше, отлично работают на статических страницах. Однако на множестве платформ используется динамический подход к генерации и загрузке контента – к примеру, для просмотра всех доступных товаров в онлайн-магазине страницу нужно не только открыть, но и прокрутить до футера. Для работы с динамическим контентом в Python нужно установить:

- Модуль [Selenium](https://pypi.org/project/selenium/).
- Драйвер [Selenium WebDriver](https://www.selenium.dev/documentation/webdriver/getting_started/install_drivers/) для браузера.
Если установка прошла успешно, выполнение этого кода приведет к автоматическому открытию страницы:

In [ ]:
from selenium import webdriver
driver = webdriver.Chrome() # или webdriver.Firefox()
driver.get('https://google.com')

Бывает, что даже после установки оптимальной версии драйвера интерпретатор Python возвращает ошибку `OSError: [WinError 216] Версия "%1" не совместима с версией Windows`. В этом случае нужно воспользоваться модулем [webdriver-manager](https://pypi.org/project/webdriver-manager/), который самостоятельно установит подходящий драйвер для нужного браузера:

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

### Имитация прокрутки страницы и парсинг данных
В качестве примера скрапинга и парсинга динамического сайта мы воспользуемся разделом [тестового онлайн-магазина](https://webscraper.io/test-sites/e-commerce/scroll/computers/tablets). Здесь расположены карточки с информацией о планшетах. Карточки загружаются ряд за рядом при прокрутке страницы:

![](https://media.proglib.io/posts/2023/03/08/8e0d31d35196e077f9c9470abf03bffd.png)

Пока страница не прокручена, полный HTML-код с информацией о планшетах получить невозможно. Для имитации прокрутки мы воспользуемся скриптом `'window.scrollTo(0, document.body.scrollHeight);'`. Цены планшетов находятся в тегах h4 класса pull-right price, а названия моделей – в тексте ссылок a класса title. Готовый код выглядит так:

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager(cache_valid_range=10).install()))
url = 'https://webscraper.io/test-sites/e-commerce/scroll/computers/tablets'
driver.get(url)
driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
time.sleep(5) 
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
prices = soup.find_all('h4', class_='pull-right price')
models = soup.find_all('a', class_='title')
for model, price in zip(models, prices):
    m = model.get_text()
    p = price.get_text()
    print(f'Планшет {m}, цена - {p}')